# CORRECTION - Exercice Agentique Météo avec Mistral AI

## Note importante

Cette correction propose **une solution parmi d'autres possibles**. Il existe plusieurs façons correctes d'implémenter ce système.

---

## Configuration préalable

Assurez-vous d'avoir :
1. Installé les dépendances : `uv add mistralai geopy requests python-dotenv`
2. Créé un fichier `.env` avec vos clés API

**Ce notebook est exécutable directement si vos clés API sont configurées.**

---

## Partie 1 : Logger

In [1]:
# logger.py
import logging

def setup_logging():
    """Configure le système de logging"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )

setup_logging()
logger = logging.getLogger(__name__)
logger.info(" Logger configuré avec succès !")

2026-02-05 23:14:26,116 - __main__ - INFO -  Logger configuré avec succès !


In [ ]:
from pathlib import Path

# Création du fichier .env
weather_key = input("Weather API Key: ")
mistral_key = input("Mistral API Key: ")

env_content = f'weather_api_key="{weather_key}"\nmistral_api_key="{mistral_key}"\n'
Path(".env").write_text(env_content)

print("✓ Fichier .env créé")

---

## Partie 2 : Modèles de données

In [3]:
# models.py
from dataclasses import dataclass

@dataclass
class UserQuery:
    question: str 
    language: str = 'French'

    def __post_init__(self):
        if self.question == "":
            raise ValueError('Must have a question')

@dataclass
class AgentiqueAnswer:
    response: str
    sources: dict

@dataclass
class Document:
    address: dict
    content: dict
    metadata: dict

# Test
query = UserQuery("Quelle est la météo ?")
logger.info(f"UserQuery créé : {query}")

2026-02-05 23:14:49,818 - __main__ - INFO - UserQuery créé : UserQuery(question='Quelle est la météo ?', language='French')


---

## Partie 3 : API Météo - CORRECTION

### Question 1.1 : Analyse du problème

**Réponse :**

Le problème majeur de la classe `WeatherAPI` originale est qu'elle **mélange la configuration (clé API) avec les données de requête (latitude, longitude)**.

**Conséquences :**
- Pour 10 adresses → 10 instances créées
- Rechargement de la clé API 10 fois (inefficace)
- Violation du principe **Single Responsibility**

**Principe SOLID violé :** Single Responsibility Principle (SRP)

### Question 1.2 : Refactoring - SOLUTION

In [4]:
# weather_api.py (VERSION CORRIGÉE)
import logging
import dotenv
import os
import requests
from dataclasses import dataclass
from geopy.geocoders import Nominatim
from typing import Tuple

dotenv.load_dotenv()
logger = logging.getLogger(__name__)


@dataclass
class GeoData:
    """Gestion de la géolocalisation d'adresses"""
    address: str
    useragent: str = "my_geocoder"
    timeout: int = 10
    locator: Nominatim = None

    def __post_init__(self):
        if self.locator is None:
            self.locator = Nominatim(user_agent=self.useragent, timeout=self.timeout)
        if self.locator is None:
            logger.error("Can't connect to geocoding agent")
            raise ConnectionError("Unable to initialize geocoder")

    def get_location(self) -> Tuple[float, float]:
        """Retourne (latitude, longitude) pour l'adresse"""
        location = self.locator.geocode(self.address)
        if location is None:
            raise ValueError(f"Address not found: {self.address}")
        return location.latitude, location.longitude


@dataclass
class WeatherConfig:
    """Configuration pour l'API météo (créée UNE SEULE FOIS)"""
    weather_api_key: str = None
    base_url: str = "https://api.openweathermap.org/data/2.5/weather"
    timeout: int = 10

    def __post_init__(self):
        if self.weather_api_key is None:
            self.weather_api_key = os.getenv('weather_api_key')
            if self.weather_api_key:
                logger.info('Weather API key loaded from environment')
        
        if self.weather_api_key is None:
            raise ValueError('Weather API key is required. Set weather_api_key in .env file')


@dataclass
class WeatherClient:
    """Client pour effectuer des requêtes météo (RÉUTILISABLE)"""
    config: WeatherConfig

    def get_weather(self, lat: float, lon: float) -> dict:
        """
        Récupère les données météo pour des coordonnées données
        
        Args:
            lat: Latitude
            lon: Longitude
            
        Returns:
            dict: Données météo au format JSON
        """
        url = self._build_url(lat, lon)
        
        try:
            response = requests.get(url, timeout=self.config.timeout)
            response.raise_for_status()
            logger.info(f"Weather data retrieved for lat={lat:.2f}, lon={lon:.2f}")
            return response.json()
        except requests.exceptions.HTTPError as e:
            logger.error(f'HTTP error occurred: {e}')
            raise
        except requests.exceptions.ConnectionError as e:
            logger.error(f'Connection error occurred: {e}')
            raise
        except requests.exceptions.Timeout as e:
            logger.error(f'Timeout error occurred: {e}')
            raise
        except Exception as e:
            logger.error(f'An unexpected exception occurred: {e}')
            raise

    def _build_url(self, lat: float, lon: float) -> str:
        """Construit l'URL de requête"""
        return f"{self.config.base_url}?lat={lat}&lon={lon}&appid={self.config.weather_api_key}"


# Test du refactoring
logger.info("""WeatherConfig et WeatherClient définis
               \n Avantages du refactoring :
                  - Configuration chargée UNE SEULE FOIS
                  - Client réutilisable pour N requêtes
                  - Séparation claire des responsabilités
                  - Facilite les tests (injection de config mockée)""")

2026-02-05 23:16:33,355 - __main__ - INFO - WeatherConfig et WeatherClient définis
               
 Avantages du refactoring :
                  - Configuration chargée UNE SEULE FOIS
                  - Client réutilisable pour N requêtes
                  - Séparation claire des responsabilités
                  - Facilite les tests (injection de config mockée)


---

## Partie 4 : Client Mistral AI - CORRECTION

In [7]:
# mistral_api.py
import os
import dotenv
from mistralai import Mistral
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import Optional

dotenv.load_dotenv()
logger = logging.getLogger(__name__)

@dataclass
class MistralSecret:
    mistral_api_key: str = None

    def __post_init__(self):
        if self.mistral_api_key is None:
            self.mistral_api_key = os.getenv('mistral_api_key')
            if self.mistral_api_key:
                logger.info(f'Mistral Key loaded : len {len(self.mistral_api_key)}')
        if self.mistral_api_key is None:
            logger.error('You must set up mistral api key')
            raise ValueError('Mistral API key is required')


@dataclass
class MistralConfig:
    temperature: float = 0
    max_tokens: int = 500
    model: str = "mistral-medium-latest"

    def __post_init__(self):
        if self.temperature < 0:
            raise ValueError('Temperature must be >=0')
        if self.max_tokens <= 0:
            raise ValueError('Max tokens must be >0')


class MistralStrategy(ABC):
    """Classe abstraite définissant le contrat pour les différentes stratégies de prompt"""
    config: MistralConfig

    @abstractmethod
    def get_prompt(self):
        """Retourne le prompt système pour cette stratégie"""
        pass

    def get_setup(self):
        """Retourne la configuration Mistral"""
        return {
            "temperature": self.config.temperature,
            "max_tokens": self.config.max_tokens,
            "model": self.config.model
        }


@dataclass
class AgenticStrategy(MistralStrategy):
    """Stratégie agentique avec contexte"""
    content: str
    config: Optional[MistralConfig] = None

    def __post_init__(self):
        if self.content == "":
            raise ValueError("you must enter content for using agentic strategy")

        if self.config is None:
            self.config = MistralConfig(
                temperature=0,
                max_tokens=500,
                model='mistral-medium-latest'
            )

    def get_prompt(self):
        return f"You're an agentic system. You will use the following content: {self.content}. " \
               "If the result is not from the content you must say it clearly."


# Question 2.3 : SimpleStrategy - SOLUTION
@dataclass
class SimpleStrategy(MistralStrategy):
    """Stratégie pour des questions simples sans contexte spécifique"""
    config: Optional[MistralConfig] = None

    def __post_init__(self):
        if self.config is None:
            self.config = MistralConfig(
                temperature=0,
                max_tokens=500,
                model='mistral-medium-latest'
            )

    def get_prompt(self) -> str:
        """Retourne le prompt système pour une assistance simple"""
        return "You are a helpful assistant. Answer questions clearly and concisely."


# Question 2.4 : MistralProvider - SOLUTION
@dataclass
class MistralProvider:
    mistralsecret: MistralSecret
    mistralconfig: MistralConfig

    def __post_init__(self):
        self._client = None

    @property
    def client(self) -> Mistral:
        """Initialise le client Mistral de manière lazy (singleton pattern)"""
        if self._client is None:
            try:
                self._client = Mistral(api_key=self.mistralsecret.mistral_api_key)
                logger.info('Mistral Client initialized')
            except ValueError as e:
                logger.error(f'Erreur: {e}')
                raise ValueError('Mistral key is not defined')
            except Exception as e:
                raise ConnectionError(f'Error {e}')
        return self._client
    
    def _create_message(self, prompt: str, strategy: MistralStrategy) -> list[dict]:
        """Crée la liste de messages pour l'API Mistral"""
        return [
            {'role': 'system', "content": strategy.get_prompt()},
            {'role': 'user', "content": prompt}
        ]
    
    def ask_mistral(self, prompt: str, context: str = "", strategy: Optional[MistralStrategy] = None):
        """
        Envoie une requête à Mistral AI
        
        Args:
            prompt: La question de l'utilisateur
            context: Le contexte pour l'agentique (optionnel)
            strategy: La stratégie à utiliser (si None, utilise AgenticStrategy avec le context)
        
        Returns:
            La réponse de Mistral
        """
        # 1. Sélection automatique de stratégie
        if strategy is None:
            if context and context.strip():
                strategy = AgenticStrategy(content=context)
                logger.info("Using AgenticStrategy (auto-selected)")
            else:
                strategy = SimpleStrategy()
                logger.info("Using SimpleStrategy (auto-selected)")
        else:
            logger.info(f"Using {strategy.__class__.__name__} (provided)")
        
        # 2. Créer les messages
        messages = self._create_message(prompt, strategy)
        
        # 3. Récupérer la configuration
        config = strategy.get_setup()
        
        # 4. Appeler l'API Mistral avec gestion d'erreurs
        try:
            chat_response = self.client.chat.complete(
                model=config['model'],
                messages=messages,
                temperature=config['temperature'],
                max_tokens=config['max_tokens']
            )
            logger.info('Successfully received response from Mistral API')
            return chat_response
            
        except Exception as e:
            logger.error(f'Mistral API error: {e}')
            raise


logger.info(" Toutes les classes Mistral définies")

# Test des stratégies
simple = SimpleStrategy()
logger.info(f"\nSimpleStrategy : {simple.get_prompt()}")
agentic = AgenticStrategy(content="Test content")
logger.info(f"AgenticStrategy : {agentic.get_prompt()[:60]}...")

2026-02-05 23:22:23,637 - __main__ - INFO -  Toutes les classes Mistral définies
2026-02-05 23:22:23,638 - __main__ - INFO - 
SimpleStrategy : You are a helpful assistant. Answer questions clearly and concisely.
2026-02-05 23:22:23,639 - __main__ - INFO - AgenticStrategy : You're an agentic system. You will use the following content...


### Questions de compréhension - RÉPONSES

**a) Pourquoi utilise-t-on `ABC` et `@abstractmethod` ?**

- Pour définir un **contrat** que toutes les stratégies doivent respecter
- Force les sous-classes à implémenter `get_prompt()`
- Permet le **polymorphisme** : utiliser n'importe quelle stratégie de la même façon
- Garantit une interface cohérente

**b) Que se passe-t-il si on tente d'instancier directement `MistralStrategy` ?**

Python lève une `TypeError` car on ne peut pas instancier une classe abstraite avec des méthodes abstraites non implémentées.

In [8]:
# Test : tentative d'instanciation de la classe abstraite
try:
    strategy = MistralStrategy()
except TypeError as e:
    logger.info(f"Erreur attendue : {e}")

2026-02-05 23:22:39,432 - __main__ - INFO - Erreur attendue : Can't instantiate abstract class MistralStrategy with abstract method get_prompt


---

## Partie 5 : Système Agentique Météo - CORRECTION

In [9]:
# agentic_weather.py - SOLUTION COMPLÈTE
from dataclasses import dataclass

@dataclass
class AgenticWeather:
    """Système agentique pour répondre à des questions sur la météo"""
    mistralprovider: MistralProvider
    weather_client: WeatherClient  # Injection de dépendance !

    def _get_context(self, address: str) -> dict:
        """Récupère les données météo pour une adresse donnée"""
        logger.info(f'Searching weather for: {address}')
        
        # 1. Géolocalisation
        userquery = UserQuery(address)
        geodata = GeoData(userquery.question)
        lat, lon = geodata.get_location()
        logger.info(f'Coordinates found: lat={lat:.2f}, lon={lon:.2f}')
        
        # 2. Requête météo avec le client réutilisable
        weather_data = self.weather_client.get_weather(lat, lon)
        
        return weather_data
    
    def ask_weather_question(self, address: str, question: str) -> str:
        """Pose une question sur la météo d'une adresse"""
        weather_data = self._get_context(address)
        context_text = self._format_weather_data(weather_data)
        
        # La stratégie est automatiquement sélectionnée dans ask_mistral
        response = self.mistralprovider.ask_mistral(question, context_text)
        
        return response.choices[0].message.content
    
    def _format_weather_data(self, data: dict) -> str:
        """Formate les données JSON de l'API météo en texte lisible"""
        try:
            weather = data['weather'][0]
            main = data['main']
            wind = data['wind']
            
            # Conversion Kelvin -> Celsius
            temp_celsius = main['temp'] - 273.15
            feels_like_celsius = main['feels_like'] - 273.15
            
            context = f"""
Données météorologiques pour {data['name']}:
- Conditions: {weather['description']}
- Température: {temp_celsius:.1f}°C (ressenti: {feels_like_celsius:.1f}°C)
- Humidité: {main['humidity']}%
- Pression: {main['pressure']} hPa
- Vent: {wind['speed']} m/s
- Visibilité: {data.get('visibility', 'N/A')} mètres
            """.strip()
            
            return context
            
        except KeyError as e:
            logger.error(f'Missing key in weather data: {e}')
            logger.debug(f'Available keys: {data.keys()}')
            return str(data)
        except Exception as e:
            logger.error(f'Error formatting weather data: {e}')
            raise


logger.info("AgenticWeather défini avec injection de WeatherClient")

2026-02-05 23:23:06,590 - __main__ - INFO - AgenticWeather défini avec injection de WeatherClient


### Question 3.1 - RÉPONSE

**Comment AgenticWeather utilise le problème de conception de WeatherAPI ?**

Dans la version originale, `AgenticWeather` créait une nouvelle instance de `WeatherAPI` à chaque appel :
```python
weather_api = WeatherAPI(lat, lon)  # Nouvelle instance à chaque fois !
```

**Avec le refactoring :**
- On injecte un `WeatherClient` réutilisable via le constructeur
- Le même client sert pour toutes les requêtes
- La configuration est chargée une seule fois
- Plus facile à tester (on peut mocker le client)

---

## Partie 6 : Tests - CORRECTION

In [11]:
# Tests pour les stratégies
import pytest

def test_agentic_strategy_with_empty_content():
    """Test que AgenticStrategy lève une erreur si content est vide"""
    with pytest.raises(ValueError, match="you must enter content"):
        AgenticStrategy(content="")
    logger.info("Test passed: AgenticStrategy rejette le contenu vide")

def test_agentic_strategy_get_prompt():
    """Test que le prompt agentique contient bien le contenu"""
    content = "Weather data for Paris"
    strategy = AgenticStrategy(content=content)
    prompt = strategy.get_prompt()
    
    assert content in prompt
    assert "agentic system" in prompt.lower()
    logger.info("Test passed: AgenticStrategy génère le bon prompt")

def test_simple_strategy_initialization():
    """Test l'initialisation de SimpleStrategy"""
    strategy = SimpleStrategy()
    
    assert strategy.config is not None
    assert strategy.config.temperature == 0
    assert strategy.config.max_tokens == 500
    logger.info("Test passed: SimpleStrategy s'initialise correctement")

def test_strategy_get_setup():
    """Test que get_setup retourne la bonne configuration"""
    strategy = SimpleStrategy()
    setup = strategy.get_setup()
    
    assert isinstance(setup, dict)
    assert "temperature" in setup
    assert "max_tokens" in setup
    assert "model" in setup
    assert setup["temperature"] == 0
    logger.info("Test passed: get_setup retourne la bonne config")

# Exécution des tests
logger.info("\nExécution des tests...\n")
test_agentic_strategy_with_empty_content()
test_agentic_strategy_get_prompt()
test_simple_strategy_initialization()
test_strategy_get_setup()
print("\nTous les tests passent !")

2026-02-05 23:23:43,611 - __main__ - INFO - 
Exécution des tests...

2026-02-05 23:23:43,612 - __main__ - INFO - Test passed: AgenticStrategy rejette le contenu vide
2026-02-05 23:23:43,612 - __main__ - INFO - Test passed: AgenticStrategy génère le bon prompt
2026-02-05 23:23:43,612 - __main__ - INFO - Test passed: SimpleStrategy s'initialise correctement
2026-02-05 23:23:43,613 - __main__ - INFO - Test passed: get_setup retourne la bonne config



Tous les tests passent !


---

## Partie 7 : Programme principal - CORRECTION

### Version complète avec gestion d'erreurs

In [14]:
def main():
    """Programme principal avec gestion d'erreurs complète"""
    logger.info("=" * 60)
    logger.info("Weather Question System - Powered by Mistral AI")
    logger.info("=" * 60)
    
    try:
        # Configuration (une seule fois)
        logger.info("Initializing system...")
        
        mistral_secret = MistralSecret()
        mistral_config = MistralConfig()
        mistral_provider = MistralProvider(mistral_secret, mistral_config)
        
        weather_config = WeatherConfig()
        weather_client = WeatherClient(weather_config)
        
        agentic_weather = AgenticWeather(
            mistralprovider=mistral_provider,
            weather_client=weather_client
        )
        
        logger.info("Système initialisé avec succès !\n")
        
        # Demander à l'utilisateur
        address = input(" Entrez une adresse : ")
        question = input(" Posez une question sur la météo : ")
        
        logger.info("\n Récupération des données et génération de la réponse...\n")
        
        # Interroger le système
        response = agentic_weather.ask_weather_question(address, question)
        
        # Afficher la réponse
        logger.info("=" * 60)
        logger.info(f" Lieu : {address}")
        logger.info(f" Question : {question}")
        logger.info("-" * 60)
        logger.info(" Réponse :")
        logger.info(response)
        logger.info("=" * 60)
        
    except ValueError as e:
        logger.info(f"\n Erreur de configuration : {e}")
        logger.info("Vérifiez votre fichier .env et vos clés API.")
        
    except ConnectionError as e:
        logger.info(f"\n Erreur de connexion : {e}")
        logger.info("Vérifiez votre connexion internet.")
        
    except Exception as e:
        logger.error(f"Unexpected error: {e}", exc_info=True)
        logger.info(f"\n Une erreur inattendue s'est produite : {e}")


# DÉCOMMENTEZ POUR EXÉCUTER LE PROGRAMME
main()

logger.info("\n Décommentez la ligne ci-dessus pour tester le système complet !")

2026-02-05 23:25:32,714 - __main__ - INFO - ============================================================
2026-02-05 23:25:32,715 - __main__ - INFO - Weather Question System - Powered by Mistral AI
2026-02-05 23:25:32,715 - __main__ - INFO - ============================================================
2026-02-05 23:25:32,716 - __main__ - INFO - Initializing system...
2026-02-05 23:25:32,716 - __main__ - INFO - Mistral Key loaded : len 32
2026-02-05 23:25:32,716 - __main__ - INFO - Weather API key loaded from environment
2026-02-05 23:25:32,717 - __main__ - INFO - Système initialisé avec succès !



 Entrez une adresse :  Paris, France
 Posez une question sur la météo :  Quel temps fait-il actuellement ?


2026-02-05 23:25:43,785 - __main__ - INFO - 
 Récupération des données et génération de la réponse...

2026-02-05 23:25:43,786 - __main__ - INFO - Searching weather for: Paris, France
2026-02-05 23:25:44,515 - __main__ - INFO - Coordinates found: lat=48.86, lon=2.32
2026-02-05 23:25:44,663 - __main__ - INFO - Weather data retrieved for lat=48.86, lon=2.32
2026-02-05 23:25:44,668 - __main__ - INFO - Using AgenticStrategy (auto-selected)
2026-02-05 23:25:44,708 - __main__ - INFO - Mistral Client initialized
2026-02-05 23:25:46,605 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-05 23:25:46,612 - __main__ - INFO - Successfully received response from Mistral API
2026-02-05 23:25:46,613 - __main__ - INFO - ============================================================
2026-02-05 23:25:46,614 - __main__ - INFO -  Lieu : Paris, France
2026-02-05 23:25:46,614 - __main__ - INFO -  Question : Quel temps fait-il actuellement ?
2026-02-05 23:2

### Version de démonstration (avec valeurs par défaut)

In [13]:
def demo():
    """Démonstration avec des valeurs prédéfinies"""
    try:
        # Configuration
        mistral_provider = MistralProvider(
            MistralSecret(),
            MistralConfig()
        )
        weather_client = WeatherClient(WeatherConfig())
        agentic_weather = AgenticWeather(mistral_provider, weather_client)
        
        # Exemple d'utilisation
        address = "Mont-Saint-Michel, France"
        question = "Quel temps fait-il actuellement ?"
        
        print(f"\n Interrogation : {address}")
        print(f" Question : {question}\n")
        
        response = agentic_weather.ask_weather_question(address, question)
        
        print("\n" + "=" * 60)
        print(" Réponse :")
        print(response)
        print("=" * 60)
        
    except Exception as e:
        print(f" Erreur : {e}")
        print("\nAssurez-vous que vos clés API sont configurées dans .env")

# DÉCOMMENTEZ POUR TESTER
demo()

2026-02-05 23:25:19,303 - __main__ - INFO - Mistral Key loaded : len 32
2026-02-05 23:25:19,304 - __main__ - INFO - Weather API key loaded from environment
2026-02-05 23:25:19,305 - __main__ - INFO - Searching weather for: Mont-Saint-Michel, France



 Interrogation : Mont-Saint-Michel, France
 Question : Quel temps fait-il actuellement ?



2026-02-05 23:25:20,006 - __main__ - INFO - Coordinates found: lat=48.64, lon=-1.51
2026-02-05 23:25:20,109 - __main__ - INFO - Weather data retrieved for lat=48.64, lon=-1.51
2026-02-05 23:25:20,112 - __main__ - INFO - Using AgenticStrategy (auto-selected)
2026-02-05 23:25:20,149 - __main__ - INFO - Mistral Client initialized
2026-02-05 23:25:21,414 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-05 23:25:21,418 - __main__ - INFO - Successfully received response from Mistral API



 Réponse :
D'après les données météorologiques pour **Huisnes-sur-Mer** que je possède :
- **Conditions** : Ciel couvert (overcast clouds)
- **Température** : 7.8°C (ressenti : 5.6°C)
- **Humidité** : 97%
- **Pression** : 979 hPa
- **Vent** : 3.41 m/s (environ 12 km/h)
- **Visibilité** : 10 000 mètres (bonne).

*Note* : Ces informations sont spécifiques à Huisnes-sur-Mer. Si vous cherchez une autre localisation, je ne peux pas répondre sans données supplémentaires.


---

## Résumé de la correction

### Concepts clés appliqués

✅ **Séparation des responsabilités** : Config vs Exécution (WeatherConfig / WeatherClient)

✅ **Classes abstraites** : MistralStrategy comme contrat

✅ **Pattern Strategy** : AgenticStrategy et SimpleStrategy interchangeables

✅ **Injection de dépendances** : WeatherClient injecté dans AgenticWeather

✅ **Lazy loading** : Client Mistral initialisé à la demande

✅ **Gestion d'erreurs** : Try/except spécifiques avec messages clairs

✅ **Logging** : Traçabilité des opérations

✅ **Tests unitaires** : Validation du comportement

### Principes SOLID respectés

- **S** : Single Responsibility - Chaque classe a une seule raison de changer
- **O** : Open/Closed - Extensible via nouvelles stratégies
- **L** : Liskov Substitution - Toutes les stratégies sont interchangeables
- **I** : Interface Segregation - Interfaces minimales
- **D** : Dependency Inversion - Dépendance sur abstractions (ABC)

### Architecture finale

```
User Input
    ↓
[main()] ← Interface utilisateur
    ↓
[AgenticWeather] ← Orchestration
    ├─ [GeoData] → Nominatim ← Lat/Lon
    ├─ [WeatherClient] → OpenWeatherMap ← Données météo
    └─ [MistralProvider] → Mistral AI ← Réponse
         └─ [Strategy] ← AgenticStrategy ou SimpleStrategy
```

---

## Points importants

### Ce qui fait une bonne solution :

1. **Réutilisabilité** : Les composants peuvent être utilisés dans d'autres contextes
2. **Testabilité** : Facile de mocker et tester chaque partie
3. **Maintenabilité** : Code clair, bien organisé, bien documenté
4. **Robustesse** : Gestion appropriée des erreurs
5. **Performance** : Pas de chargement répété de configuration

### Alternatives possibles (tout aussi valides) :

- Utilisation d'async/await pour les appels API
- Cache des résultats météo (avec TTL)
- Validation avec Pydantic au lieu de dataclasses
- Pattern Factory pour créer les stratégies
- Utilisation d'un framework de DI (dependency injection)

---

## Conclusion

Cette correction montre **une approche** parmi d'autres. L'important est de :

- Respecter les principes de conception (SOLID)
- Écrire du code lisible et maintenable
- Gérer les erreurs de manière appropriée
- Tester son code

